[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/2_train_dagger.ipynb)
# Train an Agent using the DAgger Algorithm

The DAgger algorithm is an extension of behavior cloning. 
In behavior cloning, the training trajectories are recorded directly from an expert.
In DAgger, the learner generates the trajectories but an expert corrects the actions with the optimal actions in each of the visited states.
This ensures that the state distribution of the training data matches that of the learner's current policy.

First we need an expert to learn from. For convenience we download one from the HuggingFace model hub.

In [4]:
import numpy as np
import gymnasium as gym
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(),
    n_envs=1,
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/CartPole-v0",
    venv=env,
)

c:\Users\kobbi\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
c:\Users\kobbi\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
c:\Users\kobbi\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [ ]:
import tempfile

from imitation.algorithms import bc
from imitation.algorithms.dagger import SimpleDAggerTrainer

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    rng=np.random.default_rng(),
    
)

Then we can construct a DAgger trainer und use it to train the policy on the cartpole environment.

In [2]:
import tempfile

from imitation.algorithms import bc
from imitation.algorithms.dagger import SimpleDAggerTrainer

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    rng=np.random.default_rng(),
)

with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    print(tmpdir)
    dagger_trainer = SimpleDAggerTrainer(
        venv=env,
        scratch_dir=tmpdir,
        expert_policy=expert,
        bc_trainer=bc_trainer,
        rng=np.random.default_rng(),
    )

    dagger_trainer.train(2000)

C:\Users\kobbi\AppData\Local\Temp\dagger_example_6gpjs575


Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

0batch [00:00, ?batch/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00032 |
|    entropy        | 0.32     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 86.9     |
|    loss           | 0.313    |
|    neglogp        | 0.313    |
|    prob_true_act  | 0.781    |
|    samples_so_far | 32       |
| rollout/          |          |
|    return_max     | 135      |
|    return_mean    | 97.6     |
|    return_min     | 72       |
|    return_std     | 23.5     |
--------------------------------


372batch [00:01, 195.14batch/s]


Finally, the evaluation shows, that we actually trained a policy that solves the environment (500 is the max reward).

In [3]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(dagger_trainer.policy, env, 20)
print(reward)

500.0
